In [2]:
import re
import time
import requests
import sys
import os
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# open web page

# while link anchor does not have 1994:
### scrape the link
### if not every link has 1994 then increment and go to next page. 
### if every link has 1994 end scraping. 


# https://www.congress.gov/search?q=

# https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22war%22%7D&pageSize=100&page=1
# https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22change%22%7D&pageSize=100&page=1

#https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22climate%20change%22%7D
#https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22natural%20gas%22%7D


#if year is not 1994, then increment (go to page 2)

In [47]:
def search_records(url):
    daily_editions = re.compile(r'20.*/|199[5-9]/')
    bound_editions = re.compile(r'/1990/|/1991/|/1992/|/1993/|/1994/')

    bound_edition_hits = 0
    page = 12

    urls = {}
    
    while bound_edition_hits < 100:
        page = page + 1

        #print('page' + str(page))

        request = requests.get(url + str(page))
        current_page = BeautifulSoup(request.text, 'html.parser')

        all_links_current_page = current_page.find_all('a')

        for link in all_links_current_page:
            if 'congressional-record' in link.get('href'):
                
                if re.search(bound_editions, link.get('href')):
                    bound_edition_hits = bound_edition_hits + 1

                elif re.search(daily_editions, link.get('href')):
                    result = re.search(r'(?<=").*(?=")', str(link))
                    urls['https://www.congress.gov' + str(result.group(0))] = 'url'
                    bound_edition_hits = 0

        #print('beh' + str(bound_edition_hits))

        time.sleep(.5)

    return(urls)

def scrape_records(all_urls):
    match_target_text = re.compile(r'(?<=From the Congressional Record Online through the Government Publishing Office \[www.gpo.gov\] ).*(?=____________________)', re.MULTILINE)

    target_text = {}

    for url in all_urls.keys():
        request = requests.get(url)
        current_page = BeautifulSoup(request.text, 'html.parser')

        untagged_text = current_page.body.find_all(text=True)

        for text in untagged_text:
            text = text.replace('\n',' ')
            text = " ".join(text.split())

            try:
                matched = re.search(match_target_text, text).group(0)
                target_text[matched] = url
            except:
                pass
        
        time.sleep(.5)
    
    return(target_text)

def format_records(record):
    df = pd.DataFrame(record.items(), columns=['raw', 'url'])

    df['title'] = df['raw'].str.extract(r'^(?=.+[a-z])([A-Z\W]+\b)')

    df['title'] = df['title'].str.strip()

    df['body'] = df['raw'].str.replace(r'^(?=.+[a-z])([A-Z\W]+\b)', '', regex=True)

    df['split'] = df['body'].str.split(r'([A-Z][\w]+.\s?[A-Z]+\b.)')

    df['split'] = df['split'].apply(lambda row: [x for x in row if x])

    # List items 0, 2, 4, ... are speaker names
    df['speaker_name'] = df['split'].str[::2]
    # List items 1, 3, 5, ... are spoken text
    df['speaker_text'] = df['split'].str[1::2]

    res = df[['url', 'title', 'speaker_name', 'speaker_text']].copy()

    res = res.fillna('')

    #res = res.explode(['speaker_name', 'speaker_text'])
    res = res.copy().explode('speaker_name', ignore_index = True)
    res = res.copy().explode('speaker_text', ignore_index = True)

    #res['speaker_name'] = res['speaker_name'].str.rstrip('.')

    return res

In [11]:
test = search_records('https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22croatian%22%7D&pageSize=100&page=')

In [12]:
test_2 = scrape_records(test)

In [41]:
test_3 = format_records(test_2)

In [49]:
if __name__ == '__main__':
    try:
        search_term = 'croatian'
        #search_term = sys.argv[1]
        #first_year = sys.argv[2]
        #last_year = sys.argv[3]
    except IndexError:
        exit('Congressional Data Scraper takes three arguments: search term, first year, last year. Please see github.com/stephbuon/congressional-data-scraper for more information')

    #output_folder = base_folder + '/tei_output'

    #if not os.path.exists(output_folder):
    #    os.mkdir(output_folder)

    searched = search_records('https://www.congress.gov/search?q=%7B%22source%22%3A%22congrecord%22%2C%22search%22%3A%22' + search_term + '%22%7D&pageSize=100&page=')
    scraped = scrape_records(searched)
    formatted = format_records(scraped)

    formatted.to_csv('/home/stephbuon/projects/congressional-data-scraper/congressional_records_keyword_' + search_term + '.csv')
